In [ ]:
import math

In [ ]:
with open('input.txt', 'r') as file:
    lines = file.readlines()

In [ ]:
def get_module_map():
    module_dict = {}
    
    # First pass: parse all modules
    for line in lines:
        parts = line.strip().split(' -> ')
        module_str = parts[0]
        targets = parts[1].split(', ')
        
        if module_str == 'broadcaster':
            module_dict['broadcaster'] = {
                'type': 'broadcaster',
                'targets': targets
            }
        elif module_str.startswith('%'):
            module_id = module_str[1:]
            module_dict[module_id] = {
                'type': 'flip-flop',
                'state': 0,
                'targets': targets
            }
        elif module_str.startswith('&'):
            module_id = module_str[1:]
            module_dict[module_id] = {
                'type': 'conjunction',
                'state': {},
                'targets': targets
            }

    # Second pass: find inputs for conjunctions
    for name, module in module_dict.items():
        for target in module['targets']:
            if target in module_dict and module_dict[target]['type'] == 'conjunction':
                module_dict[target]['state'][name] = 0

    return module_dict

def handle_button():
    high_sent = 0
    low_sent = 0

    module_map = get_module_map()

    for i in range(1, 1000):
        queue = [('button', 'broadcaster', 0)]

        while queue:
            source, target, pulse = queue.pop(0)
            
            if pulse == 1:
                high_sent += 1
            else:
                low_sent += 1

            if target not in module_map:
                continue
            
            module = module_map[target]
            
            if module['type'] == 'broadcaster':
                for t in module['targets']:                   
                    queue.append((target, t, pulse))
            
            elif module['type'] == 'flip-flop':
                if pulse != 0:
                    continue

                module['state'] = 1 - module['state']
                new_pulse = module['state']

                for t in module['targets']:                   
                    queue.append((target, t, new_pulse))
            
            elif module['type'] == 'conjunction':
                module['state'][source] = pulse

                if all(v == 1 for v in module['state'].values()):
                    new_pulse = 0
                else:
                    new_pulse = 1

                for t in module['targets']:
                    queue.append((target, t, new_pulse))

    return high_sent, low_sent

In [ ]:
high_sent, low_sent = handle_button()

print(f"High signals sent: {high_sent}")
print(f"Low signals sent: {low_sent}")
print(f"Product: {high_sent * low_sent}")

In [ ]:
def find_lowest_rx():
    high_sent = 0
    low_sent = 0

    module_map = get_module_map()

    # Find the module that feeds into rx
    feed = [name for name, module in module_map.items() if 'rx' in module['targets']][0]
    
    # Find all module ids that target the feed module
    rx_targets = [module_id for module_id, module in module_map.items() if feed in module.get('targets', [])]
    
    # Dictionary to store the first time we see a high pulse from each target
    cycles = {}

    i = 0

    while True:
        # Check if we found a cycle for all targets
        if len(cycles) == len(rx_targets):
            break

        i += 1

        queue = [('button', 'broadcaster', 0)]

        while queue:
            source, target, pulse = queue.pop(0)

            # We want to know when the inputs to the feed module send a HIGH pulse TO it
            if target == feed and pulse == 1:
                if source in rx_targets and source not in cycles:
                    cycles[source] = i

            if pulse == 1:
                high_sent += 1
            else:
                low_sent += 1

            if target not in module_map:
                continue
            
            module = module_map[target]
            
            if module['type'] == 'broadcaster':
                for t in module['targets']:                   
                    queue.append((target, t, pulse))
            
            elif module['type'] == 'flip-flop':
                if pulse != 0:
                    continue

                module['state'] = 1 - module['state']
                new_pulse = module['state']

                for t in module['targets']:                   
                    queue.append((target, t, new_pulse))
            
            elif module['type'] == 'conjunction':
                module['state'][source] = pulse

                if all(v == 1 for v in module['state'].values()):
                    new_pulse = 0
                else:
                    new_pulse = 1

                for t in module['targets']:
                    queue.append((target, t, new_pulse))

    return math.lcm(*cycles.values())

In [ ]:
lowest_button_press = find_lowest_rx()

print(f"Lowest button press: {lowest_button_press}")